In [7]:
import pandas as pd
business_json_path = 'data/yelp_academic_dataset_business.json'
df_b = pd.read_json(business_json_path, lines=True)

# 1 = open, 0 = closed
df_b = df_b[df_b['is_open']==1]

drop_columns = ['hours','is_open','review_count']
df_b = df_b.drop(drop_columns, axis=1)

business_RV = df_b[df_b['categories'].str.contains(
              'RV Repair|RV Dealers|RV Rental|RV Parks|Campgrounds',
              case=False, na=False)]

df_explode = df_b.assign(categories = df_b.categories
                         .str.split(', ')).explode('categories')

df_explode.categories.value_counts()

df_explode[df_explode.categories.str.contains('RV',
                      case=True,na=False)].categories.value_counts()

review_json_path = 'data/yelp_academic_dataset_review.json'
size = 1000000
review = pd.read_json(review_json_path, lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)


chunk_list = []
for chunk_review in review:
    # Drop columns that aren't needed
    chunk_review = chunk_review.drop(['review_id','useful','funny','cool'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    # Inner merge with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(business_RV, chunk_review, on='business_id', how='inner')
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)
# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

csv_name = "yelp_reviews_RV_categories.csv"
df.to_csv(csv_name, index=False)




1114 out of 1,000,000 related reviews
1124 out of 1,000,000 related reviews
999 out of 1,000,000 related reviews
1065 out of 1,000,000 related reviews
848 out of 1,000,000 related reviews
993 out of 1,000,000 related reviews
1831 out of 1,000,000 related reviews


In [ ]:
pip install numpy pandas scikit-learn nltk

In [ ]:
pip install numpy pandas scikit-learn nltk

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# Load your data
data = pd.read_csv('yelp_reviews_RV_categories.csv')
texts = data['review_text']  # This should be the name of the column containing the text
labels = data['attribute']   # This should be the name of the column with the labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)


# Creating a pipeline that first creates bag of words (after removing stopwords), then applies TF-IDF
pipeline = Pipeline([
    ('vect', CountVectorizer(stop_words=stopwords.words('english'))),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

# Train the classifier
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Print out the classification report and accuracy
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

In [ ]:
# Load your data
data = pd.read_csv('yelp_reviews_RV_categories.csv')
texts = data['review_text']  # This should be the name of the column containing the text
labels = data['attribute']   # This should be the name of the column with the labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [ ]:
# Creating a pipeline that first creates bag of words (after removing stopwords), then applies TF-IDF
pipeline = Pipeline([
    ('vect', CountVectorizer(stop_words=stopwords.words('english'))),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

In [ ]:
# Train the classifier
pipeline.fit(X_train, y_train)

In [ ]:
# Make predictions
y_pred = pipeline.predict(X_test)

# Print out the classification report and accuracy
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))